In [10]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import mode
from nltk.tokenize import sent_tokenize
from tensorflow.nn import softmax
import joblib

In [2]:
# Assuming all models have been correctly saved with their configurations and tokenizer
# For RoBERTa V3 and V4 (based on `roberta-small`)
tokenizer_small = AutoTokenizer.from_pretrained('Roberta_V3_1_task12/tokenizer')


# Load models
model = TFAutoModelForSequenceClassification.from_pretrained('Roberta_V3_task12/model')
model_check = TFAutoModelForSequenceClassification.from_pretrained('anger-disgust/model')


2024-04-02 12:37:21.461491: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-04-02 12:37:21.461775: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2403 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:c1:00.0, compute capability: 8.6
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at Roberta_V3_task12/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClas

In [3]:
# Load the test dataset
test_df = pd.read_csv('test (1).csv', sep='\t')

def prepare_data(tokenizer, sentences, max_length=128):
    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',  # PyTorch tensors
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert lists into tensors
    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)
    return input_ids, attention_masks
    
# Function to predict with the main model and return softmax scores
def predict_with_main_model(model, input_ids, attention_masks):
    predictions = model.predict([input_ids, attention_masks])[0]
    softmax_scores = softmax(predictions, axis=1)
    return softmax_scores

In [6]:
# Prepare data for prediction
test_input_ids_small, test_attention_masks_small = prepare_data(tokenizer_small, test_df['sentence'].values)

# Predict with the main model
softmax_scores = predict_with_main_model(model, test_input_ids_small, test_attention_masks_small)

45/45 [==============================] - 4s 82ms/step


In [7]:
# Create a DataFrame to hold confidence scores for each class
confidence_scores_df = pd.DataFrame(softmax_scores, columns=[f'class_{i}' for i in range(softmax_scores.shape[1])])
confidence_scores_df['id'] = test_df['id']
confidence_scores_df['sentence'] = test_df['sentence']


In [8]:
# Decision logic for sentences requiring check with the secondary model
final_predictions = []
for index, row in confidence_scores_df.iterrows():
    sorted_indices = np.argsort(-softmax_scores[index])  # Sort classes by confidence score in descending order
    if sorted_indices[0] in [0, 1] and sorted_indices[1] in [0, 1]:  # If top two predictions are anger and disgust
        # Predict with the secondary model
        secondary_pred = model_check.predict([test_input_ids_small[index:index+1], test_attention_masks_small[index:index+1]])[0]
        final_predictions.append(np.argmax(secondary_pred))
    else:
        final_predictions.append(sorted_indices[0])

1/1 [==============================] - 0s 42ms/step


In [11]:
# Convert predictions to emotion labels
label_encoder = joblib.load('Roberta_V3_1_task12/label_encoder.joblib')
predicted_emotions = label_encoder.inverse_transform(final_predictions)

In [12]:
# Save predictions with confidence scores
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'emotion': predicted_emotions
})
submission_df.to_csv('ensemble_final.csv', index=False, sep=',')

In [19]:
# Assuming `softmax_scores` contains the model predictions with shape [num_samples, num_classes]
# and `confidence_scores_df` is your DataFrame containing the test sentences and their IDs

# Map column names from 'confidence_class_{i}' to the corresponding emotion names
for i, emotion in emotion_mapping.items():
    confidence_scores_df.rename(columns={f'confidence_class_{i}': f'confidence_{emotion}'}, inplace=True)

# At this point, your confidence_scores_df will have columns like 'confidence_anger', 'confidence_disgust', etc.
# reflecting the confidence percentage for each emotion per sentence

# Optionally, if you want to add a column with the predicted emotion (the one with the highest confidence score for each sentence),
# you can do so as follows:
# Convert softmax scores to DataFrame for easier manipulation
softmax_df = pd.DataFrame(softmax_scores, columns=[emotion_mapping[i] for i in range(softmax_scores.shape[1])])

# Get the column (emotion) with the highest confidence score for each row
predicted_emotions = softmax_df.idxmax(axis=1)

# Add the predicted emotions to the original DataFrame
confidence_scores_df['predicted_emotion'] = predicted_emotions

# Now save the updated DataFrame
confidence_scores_df.to_csv('confidence_scores_with_emotion_names.csv', index=False, sep=',')

In [23]:
dfg = pd.read_csv('confidence_scores_with_emotion_names.csv')
columns_to_drop = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5']
dfg.drop(columns=columns_to_drop, inplace=True)
dfg

,id,sentence,confidence_anger,confidence_disgust,confidence_fear,confidence_happiness,confidence_sadness,confidence_surprise,predicted_emotion
0,0,Girls are happy when they get flowers,0.713853,0.098765,0.098761,98.618560,0.108955,0.361101,happiness
1,1,His jaw dropped in disbelief when he saw the p...,0.014400,0.004382,0.018674,0.013720,0.008325,99.940506,surprise
2,2,Sometimes the ugly stench makes me wanna throw...,0.292589,99.385200,0.087488,0.009502,0.198741,0.026477,disgust
3,3,The foul odor from the garbage bin was disgust...,0.009169,99.951680,0.009599,0.004701,0.008039,0.016813,disgust
4,4,"I can’t believe it, they lost the game in the ...",15.169309,1.285906,1.848264,2.969181,40.204025,38.523320,sadness
...,...,...,...,...,...,...,...,...,...
1431,1431,I feel guilty about the ice cream I stole as a...,0.460264,0.134933,0.022185,0.158591,99.206900,0.017128,sadness
1432,1432,Her ability to recall obscure facts never ceas...,0.023207,0.028145,0.005975,0.756623,0.006892,99.179160,surprise
1433,1433,"Your cat is so cute, I love him.",0.247668,0.044013,0.097881,99.332170,0.067471,0.210804,happiness
1434,1434,She remained skeptical about the new product.,0.240891,0.102229,93.074870,0.377270,0.163102,6.041641,fear


In [16]:
data_sub = 'ensemble_final.csv'

# Load the dataset
data_submission = pd.read_csv(data_sub)

# Define the mapping from integer labels to emotion names
emotion_mapping = {
    0: 'anger',
    1: 'disgust',
    2: 'fear',
    3: 'happiness',
    4: 'sadness',
    5: 'surprise'
}

# Apply the mapping to the 'emotion' column
data_submission['emotion'] = data_submission['emotion'].map(emotion_mapping)
# Save the submission file
data_submission.to_csv('ensemble_final.csv', index=False, sep=',')

In [18]:
# Load the test dataset
jkl = pd.read_csv('ensemble_final.csv')
jkl.value_counts('emotion')

emotion
happiness    284
sadness      268
disgust      255
fear         218
anger        210
surprise     201
Name: count, dtype: int64

In [24]:
from sklearn.metrics import f1_score

In [25]:
# Step 1: Load the CSV files
ensemble_df = pd.read_csv('ensemble_final.csv')
predictions_df = pd.read_csv('my_prediction.csv')

In [28]:
print(ensemble_df.columns)

Index(['id', 'emotion'], dtype='object')


In [29]:
print(predictions_df.columns)

Index(['Unnamed: 0', 'id', 'sentence', 'emotion'], dtype='object')


In [30]:
# Assuming 'emotion' in ensemble_df represents predicted emotions
predicted_emotion_col = 'emotion'  # Correct column name in ensemble_df
actual_label_col = 'emotion'  # Correct column name in predictions_df for actual labels

# Recalculate the F1 score with the corrected column names
f1 = f1_score(predictions_df[actual_label_col], ensemble_df[predicted_emotion_col], average='weighted')
print(f"F1 Score: {f1}")

# Identify the differing sentences
# Merge the predictions_df with ensemble_df on 'id' to align actual and predicted emotions alongside sentences
merged_df = predictions_df.merge(ensemble_df, on='id', suffixes=('_actual', '_predicted'))

# Filter rows where predicted emotions differ from actual emotions
diff_df = merged_df[merged_df['emotion_actual'] != merged_df['emotion_predicted']].copy()

# Number of differing sentences
num_differing = len(diff_df)
print(f"Number of sentences that differed: {num_differing}")

# Prepare a new DataFrame for differing sentences
# The diff_df already contains 'sentence' and both versions of 'emotion', so we can directly use it
# Adjust the DataFrame to only include relevant columns
diff_df = diff_df[['id', 'sentence', 'emotion_actual', 'emotion_predicted']]

# Print and save the new DataFrame with differing sentences and their predicted vs actual emotions
print(diff_df.head())
diff_df.to_csv('differing_sentences_with_predictions.csv', index=False)


F1 Score: 0.9012144459043828
Number of sentences that differed: 141
    id                                           sentence emotion_actual  \
4    4  I can’t believe it, they lost the game in the ...       surprise   
20  20  The mere thought of ghosts wandering in the at...           fear   
32  32  Receiving an unexpected gift on a regular day ...      happiness   
42  42           Seeing animals in distress makes me cry.           fear   
61  61  Discovering a dent on his car caused an immedi...          anger   

   emotion_predicted  
4            sadness  
20           disgust  
32          surprise  
42           sadness  
61           disgust  
